In [ ]:
import firebase_admin
from firebase_admin import credentials, db
from flask import Flask, request, jsonify
import os

In [ ]:
# Inisialisasi Firebase
cred = credentials.Certificate('config/firebase_sdk_admin_led_switch.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://led-switch-6c690-default-rtdb.firebaseio.com'  # Ganti dengan URL Realtime Database Anda
})

In [ ]:
app = Flask(__name__)

# Folder untuk menyimpan foto yang diterima
UPLOAD_FOLDER = './uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

In [ ]:
@app.route('/process_image', methods=['POST'])
def process_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    
    file = request.files['image']
    
    # Jika file tidak ada
    if file.filename == '':
        return jsonify({'error': 'No selected image'}), 400
    
    # Simpan file dengan nama yang unik
    file_path = os.path.join(UPLOAD_FOLDER, 'received_photo.jpg')
    file.save(file_path)
    
    # Proses gambar dan simpan ke Firebase Realtime Database
    try:
        # Mengubah status relay menjadi "on" di Firebase
        ref = db.reference('FBStatus')
        ref.set('ON')
        print("Relay status updated to 'on' in Firebase.")

        # Berikan respons sukses
        return jsonify({'message': 'Photo received successfully', 'file_path': file_path, 'relay_status': 'ON'}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/')
def index():
    return "Server is running and ready to accept images."

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)  # Jalankan server di port 5000
